# CNN_with_customdata

1.   Data의 shape = 3x64x128
2.   3 layers model
3.  학습모델 저장하기



In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [6]:
trans = transforms.Compose([
                            transforms.ToTensor()
])
train_data = torchvision.datasets.ImageFolder(root="/content/drive/My Drive/data/chair_data/train_data",transform=trans)

In [13]:
data_loader = torch.utils.data.DataLoader(dataset = train_data,
                                          batch_size=8,
                                          shuffle=True,
                                          num_workers=2,
                                          drop_last=True)

In [22]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3,6,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(6,16,kernel_size=5,stride=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2,stride=2)
        )
        self.layer3 = nn.Sequential(
            nn.Linear(16*13*29,120),
            nn.ReLU(),
            nn.Linear(120,2)
        )

    def forward(self,x):
        out = self.layer1(x)
        # print(out.shape)
        out = self.layer2(out)
        # print(out.shape)
        out = out.view(out.shape[0],-1)
        # print(out.shape)
        out = self.layer3(out)
        return out

In [21]:
# Testing 
model = CNN().to(device)
test_input = (torch.Tensor(3,3,64,128)).to(device)
test_out = model(test_input)
test_out

tensor([[ 0.0403, -0.0779],
        [ 0.0403, -0.0779],
        [ 0.0403, -0.0779]], grad_fn=<AddmmBackward>)

In [23]:
optimizer = optim.Adam(model.parameters(),lr=0.00005)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
total_batch = len(data_loader)
epochs = 3
for epoch in range(epochs):
    avg_cost = 0
    for num,data in enumerate(data_loader):
        imgs, labels = data
        imgs = imgs.to(device)
        labels = labels.to(device)        

        optimizer.zero_grad()
        hypo = model(imgs)
        cost = loss_func(hypo, labels)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch
    print('[Epoch:{}] cost = {}'.format(epoch+1, avg_cost))
print('Learning Finished!')

## 모델 저장 및 불러오기

In [ ]:
torch.save(net.state_dict(), "./model/model.pth")

In [ ]:
new_net = CNN().to(device)      # 불러올 모델과 형태가 동일해야 함
new_net.load_state_dict(torch.load('./model/model.pth'))

In [ ]:
trans = torchvision.transforms.Compose([
    transforms.Resize((64,128)),
    transforms.ToTensor()
])
test_data = torchvision.datasets.ImageFolder(root='./custom_data/test_data', transform=trans)

In [ ]:
test_set = DataLoader(dataset = test_data, batch_size = len(test_data))

In [ ]:
with torch.no_grad():
    for num, data in enumerate(test_set):
        imgs, label = data
        imgs = imgs.to(device)
        label = label.to(device)
        
        prediction = net(imgs)
        correct_prediction = torch.argmax(prediction,1) == label
        
        accuracy = correct_prediction.float().mean()
        print('Accuracy:', accuracy.item())